In [ ]:
import numpy as np
import torch


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import evaluate

In [ ]:
#The following code doesn't need import file from localdrive or googledrive.
!wget https://raw.githubusercontent.com/Zeinayyy/Citizen_Feedback_App/main/Machine_Learning/test2.xlsx 

--2023-05-31 08:07:42--  https://raw.githubusercontent.com/Zeinayyy/Citizen_Feedback_App/main/Machine_Learning/test2.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-05-31 08:07:42 ERROR 404: Not Found.



In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import TrainingArguments

In [ ]:
import pandas as pd

df = pd.ExcelFile("test2.xlsx").parse()
df.shape

(3732, 8)

In [ ]:
df.head(5)

,created_at,id,reply_count,retweet_count,favorite_count,username,clean_text,preprocessed_text,sentiment
0,Mon May 15 11:22:33 +0000 2023,1658070357826490112,0,0,1,jalan_yuuk,saat kota yang lain sibuk berlomba agar wisata...,"['kota', 'sibuk', 'berlomba', 'wisatawan', 'ko...",2
1,Mon May 15 11:43:48 +0000 2023,1658075703882170112,0,0,0,zeronol0,lol tolol lu pikir keren gitu bikin tol terus ...,"['lol', 'tolol', 'lu', 'pikir', 'keren', 'gitu...",4
2,Mon May 15 11:07:11 +0000 2023,1658066490258769920,0,0,0,PA_5060_AS,di merauke minyak tanah susah selain jalan jug...,"['merauke', 'minyak', 'tanah', 'susah', 'jalan...",5
3,Mon May 15 11:00:55 +0000 2023,1658064915532480000,0,0,1,bagrenresppu,polres ppu polda kaltim wujud kepedulian polse...,"['polres', 'ppu', 'polda', 'kaltim', 'wujud', ...",2
4,Mon May 15 11:15:36 +0000 2023,1658068608453410048,0,0,0,raksulonline,traffic light rusak jalan borong rayabatua ray...,"['traffic', 'light', 'rusak', 'jalan', 'borong...",2


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = BertForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokens = tokenizer.encode('Sampah ada di mana mana', return_tensors='pt')

result = model(tokens)

In [ ]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-0.5956, -0.3068, -0.1598,  0.2065, -0.1191]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
result.logits

tensor([[-0.5956, -0.3068, -0.1598,  0.2065, -0.1191]],
       grad_fn=<AddmmBackward0>)

In [ ]:
int(torch.argmax(result.logits))+1

4

In [ ]:
import joblib
joblib.dump(result, "BERT.joblib")
joblib.dump(tokens, "Tokenizer.joblib")

['Tokenizer.joblib']

In [ ]:
df['clean_text'].iloc[63]

'prioritas gubernur andi sudirman pemprov tangani jalan rusak di yasin limpo gowa httpstco lnvqnop httpstcos sro snb'

In [ ]:
def sentiment_score(clean_text):
    tokens = tokenizer.encode(clean_text, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['clean_text'].iloc[10])

4

In [ ]:
df['sentiment'] = df['clean_text'].apply(lambda x: sentiment_score(str(x)))

In [ ]:
feature = df['preprocessed_text']

In [ ]:
label = df['sentiment']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature,
                                                    label,
                                                    random_state = 100,
                                                    test_size = 0.2)

In [ ]:
train_args = TrainingArguments(output_dir="test")

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
  logits, label = eval_pred
  predictions = np.argmax(logits, axis=1)
  return metrics.compute(predictions=predictions, references=label)

In [ ]:
from transformers import Trainer

In [ ]:
train_args = TrainingArguments(output_dir="test",
                               evaluation_strategy="epoch",
                              optim="adamw_torch")

In [ ]:
trainer = Trainer(
    model = model,
    args = train_args,
    train_dataset = X_train,
    eval_dataset = X_test,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py:3802 in get_loc              │
│                                                                                                  │
│   3799 │   │   │   │   )                                                                         │
│   3800 │   │   │   casted_key = self._maybe_cast_indexer(key)                                    │
│   3801 │   │   │   try:                                                                          │
│ ❱ 3802 │   │   │   │   return self._engine.get_loc(casted_key)                                   │
│   3803 │   │   │   except KeyError as err:                                                       │
│   3804 │   │   │   │   raise KeyError(key) from err                                              │
│   3805 │   │   │   except TypeError:                                                             │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:138                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:165                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.Int64HashTable.get_item:2263                                           │
│                                                                                                  │
│ in pandas._libs.hashtable.Int64HashTable.get_item:2273                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 314

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1909 in _inner_training_loop     │
│                                                                                                  │
│   1906 │   │   │   │   rng_to_sync = True                                                        │
│   1907 │   │   │                                                                                 │
│   1908 │   │   │   step = -1                                                                     │
│ ❱ 1909 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1910 │   │   │   │   total_batched_samples += 1                          